In [1]:
import os

import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BartForConditionalGeneration, BartTokenizer
import shap

In [2]:
GPU = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU

if torch.cuda.is_available():    
    print(torch.cuda.current_device())
    print(torch.cuda.device)
    torch.cuda.device = GPU
    print(torch.cuda.device)
    device = 'cuda'
else:
    device = 'cpu'
print(f'device: {device}')

0
<class 'torch.cuda.device'>
1
device: cuda


In [3]:
model_path = "/srv/scratch6/kew/ats/muss/resources/models/muss_en_mined_hf"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = BartForConditionalGeneration.from_pretrained(model_path, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

model = model.eval()
model = model.to(device)

In [4]:
# modifying the value of a single special token
sentences = [
    "<DEPENDENCYTREEDEPTHRATIO_0.0> <WORDRANKRATIO_0.75> <REPLACEONLYLEVENSHTEIN_0.65> <LENGTHRATIO_0.75> This is extremely hard to comprehend.",
    "<DEPENDENCYTREEDEPTHRATIO_0.2> <WORDRANKRATIO_0.75> <REPLACEONLYLEVENSHTEIN_0.65> <LENGTHRATIO_0.75> This is extremely hard to comprehend.",
    "<DEPENDENCYTREEDEPTHRATIO_0.4> <WORDRANKRATIO_0.75> <REPLACEONLYLEVENSHTEIN_0.65> <LENGTHRATIO_0.75> This is extremely hard to comprehend.",
    "<DEPENDENCYTREEDEPTHRATIO_0.6> <WORDRANKRATIO_0.75> <REPLACEONLYLEVENSHTEIN_0.65> <LENGTHRATIO_0.75> This is extremely hard to comprehend.",
    "<DEPENDENCYTREEDEPTHRATIO_0.8> <WORDRANKRATIO_0.75> <REPLACEONLYLEVENSHTEIN_0.65> <LENGTHRATIO_0.75> This is extremely hard to comprehend.",
    "<DEPENDENCYTREEDEPTHRATIO_1.0> <WORDRANKRATIO_0.75> <REPLACEONLYLEVENSHTEIN_0.65> <LENGTHRATIO_0.75> This is extremely hard to comprehend.",
]
batch = tokenizer(sentences, padding=True, return_tensors="pt")

generated_ids = model.generate(batch["input_ids"].to(model.device))
generated_sents = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
for s in generated_sents:
    print(s)

This is very hard to understand..
This is very hard to understand..
This is very hard to understand..
This is very hard to understand..
This is very hard to understand..
This is very hard to understand..


In [5]:
# shap: https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/translation/Machine%20Translation%20Explanations.html
explainer = shap.Explainer(model, tokenizer)

In [8]:
shap_values = explainer(sentences[:1])

  0%|          | 0/248 [00:00<?, ?it/s]

In [ ]:
shap.initjs()
shap.plots.text(shap_values)